<a href="https://colab.research.google.com/github/madhavamk/computational-data-science/blob/master/MiniProjects/M5_NB_MiniProject_1_Medical_Q%26A_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A programme by IISc and TalentSprint
### Mini-Project: Medical Q&A using GPT2

## Learning Objectives

At the end of the experiment, you will be able to:

* perform data preprocessing, EDA and feature extraction on the Medical Q&A dataset
* load a pre-trained tokenizer
* finetune a GPT-2 language model for medical question-answering

## Dataset Description

The dataset used in this project is the *Medical Question Answering Dataset* ([MedQuAD](https://github.com/abachaa/MedQuAD/tree/master)). It includes medical question-answer pairs along with additional information, such as the question type, the question *focus*, its UMLS(Unified Medical Language System) details like - Concept Unique Identifier(*CUI*) and Semantic *Type* and *Group*.

To know more about this data's collection, and construction method, refer to this [paper](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-3119-4).

The data is extracted and is in CSV format with below features:

- **Focus**: the question focus
- **CUI**: concept unique identifier
- **SemanticType**
- **SemanticGroup**
- **Question**
- **Answer**

## Part-A: Grading = 10 Points

## Information

Healthcare professionals often have to refer to medical literature and documents while seeking answers to medical queries. Medical databases or search engines are powerful resources of upto date medical knowledge. However, the existing documentation is large and makes it difficult for professionals to retrieve answers quickly in a clinical setting. The problem with search engines and informative retrieval engines is that these systems return a list of documents rather than answers. Instead, healthcare professionals can use question answering systems to retrieve short sentences or paragraphs in response to medical queries. Such systems have the biggest advantage of generating answers and providing hints in a few seconds.

### Problem Statement

Fine-tune gpt2 model on medical-question-answering-dataset for performing response generation for medical queries.

Please refer to ***M6 Assignment-1 Fine-tune GPT2*** to get familiar with how to load pre-trained gpt2 tokenizer and model.

### Import required packages

In [1]:
!pip -q install -U accelerate
!pip -q install -U transformers
!pip -q install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.7 MB/s eta 0:00:00


In [2]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

import warnings
warnings.filterwarnings('ignore')

In [3]:
#@title Download the dataset
!wget -q https://cdn.iisc.talentsprint.com/AIandMLOps/MiniProjects/Datasets/MedQuAD.csv
!ls | grep ".csv"

MedQuAD.csv


**Exercise 1: Read the MedQuAD.csv dataset**

**Hint:** pd.read_csv()

In [4]:
df = pd.read_csv("MedQuAD.csv")
df.shape

(16412, 6)

In [5]:
df.head()

,Focus,CUI,SemanticType,SemanticGroup,Question,Answer
0,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What is (are) Adult Acute Lymphoblastic Leukem...,Key Points - Adult acute lymphoblastic leukemi...
1,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What are the symptoms of Adult Acute Lymphobla...,"Signs and symptoms of adult ALL include fever,..."
2,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,How to diagnose Adult Acute Lymphoblastic Leuk...,Tests that examine the blood and bone marrow a...
3,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What is the outlook for Adult Acute Lymphoblas...,Certain factors affect prognosis (chance of re...
4,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,Who is at risk for Adult Acute Lymphoblastic L...,Previous chemotherapy and exposure to radiatio...


### Pre-processing and EDA

**Exercise 2: Perform below operations on the dataset [0.5 Mark]**

- Handle missing values
- Remove duplicates from data considering `Question` and `Answer` columns

- **Handle missing values**

In [6]:
# YOUR CODE HERE
df.isnull().sum()

,0
Focus,14
CUI,565
SemanticType,597
SemanticGroup,565
Question,0
Answer,5


In [7]:
# Drop missing values
# YOUR CODE HERE
df.dropna(inplace=True)


- **Remove duplicates from data considering `Question` and `Answer` columns**

In [9]:
# Check duplicates
# YOUR CODE HERE
df.duplicated(subset=['Question','Answer']).sum()

48

In [11]:
# Drop duplicates
# YOUR CODE HERE
df.drop_duplicates(subset=['Question','Answer'],inplace=True)

In [12]:
# Check duplicates
# YOUR CODE HERE
df.duplicated(subset=['Question','Answer']).sum()

0

**Exercise 3: Display the category name, and the number of records belonging to top 100 categories of `Focus` column [1 Mark]**

In [13]:
# YOUR CODE HERE
df['Focus'].value_counts()[:100]

,count
Focus,
Breast Cancer,53
Prostate Cancer,43
Stroke,35
Skin Cancer,34
Alzheimer's Disease,30
...,...
Alzheimer's Caregiving,11
Polycythemia Vera,11
"Diabetes, Heart Disease, and Stroke",11


In [15]:
# Top 100 Focus categories names
# YOUR CODE HERE
categories = df['Focus'].value_counts()[:100].index.tolist()

### Create Training and Validation set

**Exercise 4: Create training and validation set [2 Marks]**

- Consider 4 samples per `Focus` category, for each top 100 categories, from the dataset (It will give 400 samples for training)

- Consider 1 sample per `Focus` category (different from training set), for each top 100 categories, from the dataset (It will give 100 samples for validation)

In [16]:
# YOUR CODE HERE
df_train = pd.DataFrame(columns=['Focus', 'Question', 'Answer'])
df_copy = df[['Focus', 'Question', 'Answer']]
df_val = pd.DataFrame(columns=['Focus', 'Question', 'Answer'])
for i in categories:
    #print(df_copy.query('Focus == @i').sample(n=4))
    df_train = pd.concat([df_train, df_copy.query('Focus == @i').sample(n=4)])
    df_val = pd.concat([df_val, df_copy.drop(df_train.index).query('Focus == @i').sample(n=1)])

### Pre-process `Question` and `Answer` text

**Exercise 5: Perform below tasks: [1.5 Marks]**

- Combine `Question` and `Answer` for train and validation data as shown below:
    - sequence = *'\<question\>' + question-text + '\<answer\>' + answer-text*

- Join the combined text using '\n' into a single string for training and validation separately

- Save the training and validation strings as separate text files

- **Combine Question and Answer for train and val data**

In [17]:
# YOUR CODE HERE
df_train['sequence'] = '<question>' + df_train['Question'] + '<answer>' + df_train['Answer']
df_val['sequence'] = '<question>' + df_val['Question'] + '<answer>' + df_val['Answer']

- **Join the combined text using '\n' into a single string for training and validation separately**

In [18]:
# YOUR CODE HERE
train_text = '\n'.join(df_train['sequence'])
val_text = '\n'.join(df_val['sequence'])

- **Save the training and validation strings as text files**

In [19]:
# YOUR CODE HERE
with open('train.txt', 'w') as f:
    f.write(train_text)

with open('val.txt', 'w') as f:
    f.write(val_text)

**Exercise 6: Load pre-trained GPT2Tokenizer [0.5 Mark]**

- Use checkpoint = "gpt2"

In [20]:
# YOUR CODE HERE
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

**Exercise 7: Tokenize train and validation data and form TextDataset objects [0.5 Mark]**

- Use the loaded pre-trained tokenizer
- Use training and validation data saved in text files

In [22]:
# YOUR CODE HERE
train_dataset = TextDataset(tokenizer=tokenizer, file_path='train.txt', block_size=128)
val_dataset = TextDataset(tokenizer=tokenizer, file_path='val.txt', block_size=128)

**Exercise 8: Create a DataCollator object [0.5 Mark]**

In [23]:
# YOUR CODE HERE
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

**Exercise 9: Load pre-trained GPT2LMHeadModel [0.5 Mark]**

In [24]:
# YOUR CODE HERE
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**Exercise 10: Fine-tune GPT2 Model [1 Mark]**

- Specify training arguments and create a TrainingArguments object (Use 30 epochs)

- Train a GPT-2 model using the provided training arguments

- Save the resulting trained model and tokenizer to a specified output directory

In [25]:
# Set up the training arguments
model_output_path = "/content/gpt_model"

# YOUR CODE HERE
training_args = TrainingArguments(
    output_dir=model_output_path,
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps=400,
    save_steps=800,
    warmup_steps=5,
    prediction_loss_only=True,
)

In [26]:
# Train the model
# YOUR CODE HERE
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
# Save the model
# YOUR CODE HERE
trainer.train()
trainer.save_model()

# Save the tokenizer
# YOUR CODE HERE
tokenizer.save_pretrained(model_output_path)

Step,Training Loss
500,2.549900
1000,2.001700
1500,1.640700
2000,1.366000
2500,1.139000
3000,0.949900
3500,0.805500
4000,0.677800
4500,0.585200
5000,0.507100


('/content/gpt_model/tokenizer_config.json',
 '/content/gpt_model/special_tokens_map.json',
 '/content/gpt_model/vocab.json',
 '/content/gpt_model/merges.txt',
 '/content/gpt_model/added_tokens.json')

**Exercise 11: Test Model with user input prompts [1 Mark]**

- Create `generate_response()` function that takes a trained *model*, *tokenizer*, and a *prompt* string as input and generates a response using the GPT-2 model

- Test it with some user input prompts

In [27]:
# YOUR CODE HERE
def generate_response(model, tokenizer, prompt, max_length=100):

    input_ids = tokenizer.encode(prompt, return_tensors="pt")      # 'pt' for returning pytorch tensor

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [29]:
# Load the fine-tuned model and tokenizer

# YOUR CODE HERE
my_model = GPT2LMHeadModel.from_pretrained(model_output_path)
my_tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

In [30]:
# Response from model

# YOUR CODE HERE
prompt = "Who is at risk for Colorectal Cancer?"  # Replace with your desired prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: Who is at risk for Colorectal Cancer? ?<answer>Certain factors can increase or decrease the risk of colorectal cancer. Anything that increases your chance of getting a disease is called a risk factor. Anything that increases your chance of getting a disease is called a protective factor.
<question>What are the treatments for Colorectal Cancer ?<answer>Researchers have not found that eating, diet, and nutrition play a role in causing or preventing colorectal


In [31]:
# Testing with given prompt 1

# YOUR CODE HERE
prompt = "How to diagnose Breast Cancer"  # Replace with your desired prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: How to diagnose Breast Cancer ?<answer>Your doctor will diagnose breast cancer based on your symptoms, your age and overall health, your medical history, a physical exam, and test results. He or she will look for cancer cells in your bone marrow, in your lymph nodes, in your pancreas, and in nearby tissues such as your breast. Your doctor also will try to rule out other possible causes of your symptoms. Medical History Your doctor may ask you detailed questions about your medical history.


In [32]:
# Testing with given prompt 2

# YOUR CODE HERE
prompt = "Who is at risk for Breast Cancer?"  # Replace with your desired prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: Who is at risk for Breast Cancer? Breast cancer occurs most often in women who are pregnant or have just given birth. The risk increases with gestational age, but increases with age. Women who have breast cancer are at increased risk for having another child. Breast cancer occurs most often in women who are pregnant or have just given birth. The risk increases with gestational age, but increases with age. (Watch the video to learn more about breast cancer. To enlarge the video, click the brackets in


**Exercise 12: Compare the performance of a *GPT2 model* with the *GPT2 model fine-tuned* on MedQuAD data [1 Mark]**

- Load another pre-trained GPT2LMHeadModel and do not fine-tune it

- To generate response using the untuned model, pass it as a parameter to `generate_response()` function

- Test both models (fine-tuned and untuned) with below user input prompts:

    - "What precautions to take for a healthy life?"
    - "What to do after being diagnosed with cancer?"
    - "What to do when feeling sick?"

In [35]:
# Load a pre-trained GPT2 model, do not finetune it with MedQuAD data

# YOUR CODE HERE
untuned_model = GPT2LMHeadModel.from_pretrained('gpt2')

In [36]:
# Testing with finetuned model: prompt 1

# YOUR CODE HERE
prompt = "What precautions to take for a healthy life?"  # Replace with your desired prompt
response = generate_response(my_model, tokenizer, prompt)
print("Generated response:", response)

Generated response: What precautions to take for a healthy life? - What is the name of my blood pressure medication? - Is that the brand name or the generic name? - What are the possible side effects of my medication? (Be sure the doctor knows about any allergies you have and any other medications you are taking, including over-the-counter drugs, vitamins, and dietary supplements.) - What time of day should I take my blood pressure medicine? - Are there any foods, beverages, or dietary supplements I


In [37]:
# Testing with untuned model: prompt 1

# YOUR CODE HERE
prompt = "What precautions to take for a healthy life?"  # Replace with your desired prompt
response = generate_response(untuned_model, tokenizer, prompt)
print("Generated response:", response)

Generated response: What precautions to take for a healthy life?

The following are some of the most common questions you'll hear from your doctor or nurse about your health.

What are the risks of taking a drug that can cause cancer?

The risks of taking a drug that can cause cancer are very high.

What are the risks of taking a drug that can cause cancer?

The risks of taking a drug that can cause cancer are very high.

What are the risks


In [38]:
# Testing with finetuned model: prompt 2

# YOUR CODE HERE
prompt = "What to do after being diagnosed with cancer?"  # Replace with your desired prompt
response = generate_response(my_model, tokenizer, prompt)
print("Generated response:", response)

Generated response: What to do after being diagnosed with cancer?
<question>What are the treatments for Pituitary Tumors ?<answer>Treatment for pituitary tumors depends on the type and stage of the tumor. Pituitary tumors are generally diagnosed in the early stages of the disease and generally show signs and symptoms of the disease. Early diagnosis and treatment of the tumor type are key to successful treatment.
<question>What is (are) Pituitary Tumors ?<


In [39]:
# Testing with untuned model: prompt 2

# YOUR CODE HERE
prompt = "What to do after being diagnosed with cancer?"  # Replace with your desired prompt
response = generate_response(untuned_model, tokenizer, prompt)
print("Generated response:", response)

Generated response: What to do after being diagnosed with cancer?

The first step is to get your doctor's approval for a treatment.

If you have a cancer diagnosis, you may need to get a second opinion.

If you have a cancer diagnosis, you may need to get a second opinion. If you have a cancer diagnosis, you may need to get a third opinion.

If you have a cancer diagnosis, you may need to get a third opinion. If you have a cancer


In [40]:
# Testing with finetuned model: prompt 3

# YOUR CODE HERE
prompt = "What to do when feeling sick?"  # Replace with your desired prompt
response = generate_response(my_model, tokenizer, prompt)
print("Generated response:", response)

Generated response: What to do when feeling sick? When to see a doctor? When should I talk to my family about dialysis or a kidney transplant? What is the name of my blood pressure medication? Is that the brand name or the generic name? What are the possible side effects of my medication? (Be sure the doctor knows about any allergies you have and any other medications you are taking, including over-the-counter drugs, vitamins, and dietary supplements.) What time of day should I take my blood


In [41]:
# Testing with untuned model: prompt 3

# YOUR CODE HERE
prompt = "What to do when feeling sick?"  # Replace with your desired prompt
response = generate_response(untuned_model, tokenizer, prompt)
print("Generated response:", response)

Generated response: What to do when feeling sick?

The first thing you should do is to get your body to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you should take a few minutes to relax.

If you're feeling sick, you
